In [1]:
%matplotlib widget

import math, numba, pprint, pylab
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets

from itertools import islice

from ipywidgets import (
    interact,
    interact_manual,
    interactive,
    fixed,
    FloatSlider,
    Dropdown,
    Layout
)

from ipywidgets import (Output, HBox, VBox)

pylab.rcParams['figure.figsize'] = (12.0, 6.0)
plt.style.use('dark_background')

In [2]:
slider_layout=Layout(width='66%')

In [3]:
def tg_pprint(*args, width=200, **kwargs):
    return pprint.pprint(*args, **kwargs, width=width)

In [4]:
@numba.njit
def log_map_gen(x0=.5, r=2.4):
    xn1=x0
    while True:
        yield xn1
        xn=xn1
        xn1 = r*xn*(1.-xn)

In [5]:
@interact(   x0=FloatSlider(.5, min=0., max=1., step=.001, layout=slider_layout)
            ,r= FloatSlider(2.98, min=0., max=4., step=.001, layout=slider_layout)
            ,nthrow=Dropdown(options=(0, 1, 2, 3, 4, 10, 20, 40, 80, 160, 320, 640)) # nthrow=fixed(50)
            ,nkeep= Dropdown(options=(   10, 20, 30, 40, 50), index=1)
            ,threeToFour=False
#            ,norm=fixed(None)
            ,norm=fixed(lambda x: int(1000*x)/10)
         )
def print_pop(x0=.5, r=2.4, nthrow=0, nkeep=30, threeToFour=False, norm=lambda x: int(1000*x)/10):
    if threeToFour:
        r = 3 + r/4
    seq=islice(log_map_gen(x0=x0, r=r), nthrow, nthrow+nkeep)
    if norm:
        seq=map(norm, seq)
    tg_pprint(tuple(seq))

interactive(children=(FloatSlider(value=0.5, description='x0', layout=Layout(width='66%'), max=1.0, step=0.001…

In [6]:
plt.close()
@interact(    n=Dropdown(options=[32, 64, 128, 256, 512, 1024], index=2)
#            ,n=[('30', 30), ('100', 100), ('200', 200)] # n=fixed(50)
            ,x0=FloatSlider(.5, min=0., max=1., step=.001, layout=slider_layout)
            ,r= FloatSlider(2.98, min=0., max=4., step=.001, layout=slider_layout)
            ,threeToFour=False
         )
def graph_pop(n=30, x0=.5, r=2.4, threeToFour=False):
    if threeToFour:
        r = 3 + r/4
    ts = range(n)
    xs = tuple(islice(log_map_gen(x0=x0, r=r), n))
    #tg_pprint(xs)
    plt.gcf().clear()
    plt.ylim((0, 1.))
    plt.plot(ts, xs, marker=',')

interactive(children=(Dropdown(description='n', index=2, options=(32, 64, 128, 256, 512, 1024), value=128), Fl…

In [15]:
@numba.njit
def gen_rs(rmin=1., rmax=4., nbins=2000):
    delta = rmax - rmin
    for i in range(nbins):
        yield rmin + delta * i / nbins

In [27]:
@numba.njit
def gen_bifurication_map(rmin=0, rmax=4, nthrow=50, nkeep=50, nbins=2000):
    rs = []
    xs = []
    for r in gen_rs(rmin=rmin, rmax=rmax, nbins=nbins):
        lm_gen = log_map_gen(r=r)

        for _ in range(nthrow):
            next(lm_gen)

        for _ in range(nkeep):
            rs.append(r)
            xs.append(next(lm_gen))

        #x = tuple(islice(log_map_gen(r=r), nthrow, nthrow+nkeep))
        #rs.append(len(x)*(r,))
        #xs.append(x)
    return rs, xs

In [40]:
@numba.guvectorize(['void(float32[:], int32, float32[:])'], '(n),()->(n)')
def gen_bifurication_map_v(r, nthrow, xs):
    lm_gen = log_map_gen(r=r)
    
    for _ in range(nthrow):
        next(lm_gen)

    for i in range(xs.size):
        xs[i] = next(lm_gen)

NameError: undefined output symbols: n

In [28]:
def foobar(rmin_=0, rmax_=4.0):
    #plt.close()
    @interact_manual(    nthrow=Dropdown(options=(0, 1, 2, 3, 4, 10, 20, 40, 80, 160, 320, 640), index=7) # nthrow=fixed(50)
                        ,nkeep= Dropdown(options=(   1, 2, 3, 4, 10, 20, 40, 80, 160, 320, 640), index=6)
                        ,nbins= Dropdown(options=(8000, 16000, 32000, 64000, 128000, 256000, 512000), index=3)
                        ,rmin=FloatSlider(rmin_, min=rmin_, max=rmax_, step=0.0001, layout=slider_layout)
                        ,rmax=FloatSlider(rmax_, min=rmin_, max=rmax_, step=0.0001, layout=slider_layout)
                    )
    def plot_bifurication_map(rmin=0, rmax=4, nthrow=50, nkeep=50, nbins=2000):
        #rs = []
        #xs = []
        #for r in gen_rs(rmin=rmin, rmax=rmax, nbins=nbins):
        #    x = tuple(islice(log_map_gen(r=r), nthrow, nthrow+nkeep))
        #    rs.append(len(x)*(r,))
        #    xs.append(x)
        rs, xs = gen_bifurication_map(rmin=rmin, rmax=rmax, nthrow=nthrow, nkeep=nkeep, nbins=nbins)
        print(f'nrs:{len(rs)} nxs:{len(xs)} type rs:{type(rs)} xs:{type(xs)}')
        #plt.yscale('linear')
        #plt.gcf().clear()
        plt.plot(tuple(rs), tuple(xs), marker='.', linewidth=0, markersize=.2)
        tg_pprint(rs[:30])
        tg_pprint(xs[:30])

In [29]:
foobar(0.75, 4)
#foobar(3.64, 3.74)
#foobar(2.9, 3.1)

interactive(children=(FloatSlider(value=0.75, description='rmin', layout=Layout(width='66%'), max=4.0, min=0.7…

In [11]:
plt.close()
plt.figure()
import matplotlib.widgets

hbox = HBox()

def graph_pop(n=30, x0=.5, r=2.4, threeToFour=False, out=hbox):
    with out:
        if threeToFour:
            r = 3 + r/4
        ts = range(n)
        xs = tuple(islice(log_map_gen(x0=x0, r=r), n))
        #tg_pprint(xs)
        plt.gcf().clear()
        plt.ylim((0, 1.))
        plt.plot(ts, xs, marker=',')

#w = interactive( graph_pop, n=Dropdown(options=[32, 64, 128, 256, 512, 1024])
#            ,n=[('30', 30), ('100', 100), ('200', 200)] # n=fixed(50)
#            ,x0=FloatSlider(.5, min=0., max=1., step=.001)
#            ,r= FloatSlider(2.98, min=0., max=4., step=.001)
#            ,threeToFour=False
#         )

#hbox.children += (w,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
#import bokeh.plotting as plotting

#plotting.output_notebook()

#f = plotting.figure(plot_width=600, plot_height=400)
#f.circle(*zip(*gen_r_e(nbins=500000)),size=.5)
#plotting.show(f)

In [13]:
def gen_mandel(xrng=(-2,.5), yrng=(-1,1), horizon=2, nbins=100, niter=32):
    rng = ((xrng[1] - xrng[0])
          ,(yrng[1] - yrng[0])
          )
    delta = tuple(map(lambda v: v/nbins, rng))

    xs = np.linspace(*xrng, nbins, dtype=float)
    ys = np.linspace(*yrng, nbins, dtype=float)
    cs = xs + ys[:, None]*1j
    ns = np.zeros(cs.shape, dtype=int)
    zs = np.zeros(cs.shape, dtype=np.complex64)
    m  = np.zeros((nbins, nbins), dtype=complex)
    
    for it in range(niter):
        i = np.less(abs(zs), horizon)
        ns[i] = it
        zs[i] = zs[i]**2 + cs[i]
    ns[ns == niter - 1] = 0
    
    log_horizon = np.log(np.log(horizon))/np.log(2)
    with np.errstate(invalid='ignore'):
        zs = np.nan_to_num(ns + 1 - np.log(np.log(abs(zs))/np.log(2) + log_horizon))
    return zs, ns, cs

In [14]:
zs, ns, cs = gen_mandel(nbins=1000, horizon=2**10)
#plt.countour(xs, ys, np.absolute(m))
plt.imshow(np.absolute(zs))
#tg_pprint(zs)

In [15]:
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.view_init(azim=-90, elev=90)
ax.scatter3D(cs.real, cs.imag, abs(zs), c=abs(zs).flat, marker='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
abs(2+-1.324j)

2.398536220281028

In [17]:
print(np.finfo(np.float64))

Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   eps =        2.2204460492503131e-16
negep =     -53   epsneg =     1.1102230246251565e-16
minexp =  -1022   tiny =       2.2250738585072014e-308
maxexp =   1024   max =        1.7976931348623157e+308
nexp =       11   min =        -max
---------------------------------------------------------------

